In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import json

import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [61]:
#lesum inn gögn af epl offical og understat
import pickle

pick_read = open('drive/My Drive/Salka ML football/data/understat_epl_ready.pickle','rb')
understat_epl_ready = pickle.load(pick_read)
pick_read.close()

pick_read = open('drive/My Drive/Salka ML football/data/Epl_official_ready.pickle','rb')
Epl_official_ready = pickle.load(pick_read)
pick_read.close()


In [5]:
#lesum inn odds af football-data.co.uk
csvs = ['https://www.football-data.co.uk/mmz4281/1718/E0.csv',
        'https://www.football-data.co.uk/mmz4281/1819/E0.csv',
        'https://www.football-data.co.uk/mmz4281/1920/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2021/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2122/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2223/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2324/E0.csv',
        'https://www.football-data.co.uk/mmz4281/2425/E0.csv']
odds = pd.DataFrame()
for c in csvs:
    odds = pd.concat([odds, pd.read_csv(c)])
odds.to_csv("football_odds.csv.gzip", index=False, compression='gzip') # save combined file
odds.shape

(3040, 153)

In [6]:
odds_df = odds[['Date','HomeTeam','AwayTeam', 'HTR','HS', 'AS', 'HST', 'AST', 'HC', 'AC',
                'HF','AF', 'B365H','B365D','B365A' ]]
odds_df.columns = ['Date','Home_Team','Away_Team', 'Half Time Result','HomeShots','AwayShots','HomeShotOT','AwayShotsOT',
                   'HomeFouls','AwayFouls','HomeCorners', 'AwayCorners','OddsH','OddsD','OddsA']
odds_df.head()

,Date,Home_Team,Away_Team,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA
0,11/08/2017,Arsenal,Leicester,D,27,6,10,3,9,4,9,12,1.53,4.5,6.50
1,12/08/2017,Brighton,Man City,D,6,14,2,4,3,10,6,9,11.00,5.5,1.33
2,12/08/2017,Chelsea,Burnley,A,19,10,6,5,8,5,16,11,1.25,6.5,15.00
3,12/08/2017,Crystal Palace,Huddersfield,A,14,8,4,6,12,9,7,19,1.83,3.6,5.00
4,12/08/2017,Everton,Stoke,H,9,9,4,1,6,7,13,10,1.70,3.8,5.75


In [62]:
#skoða hvort lið séu með mismunandi nöfn milli gagnasafna
a = (odds_df.Home_Team.unique())
b = (understat_epl_ready.Home_Team.unique())
c = (Epl_official_ready.Home_Team.unique())
print(sorted(a))
print(sorted(b))
print(sorted(c))

['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield', 'Ipswich', 'Leeds', 'Leicester', 'Liverpool', 'Luton', 'Man City', 'Man United', 'Newcastle', 'Norwich', "Nott'm Forest", 'Sheffield United', 'Southampton', 'Stoke', 'Swansea', 'Tottenham', 'Watford', 'West Brom', 'West Ham', 'Wolves']
['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield', 'Ipswich', 'Leeds', 'Leicester', 'Liverpool', 'Luton', 'Manchester City', 'Manchester United', 'Newcastle United', 'Norwich', 'Nottingham Forest', 'Sheffield United', 'Southampton', 'Stoke', 'Swansea', 'Tottenham', 'Watford', 'West Bromwich Albion', 'West Ham', 'Wolverhampton Wanderers']
['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield', 'Ipswich', 'Leeds', '

In [63]:
print(len(c))

30


In [64]:
replace_values = {'Man City' : 'Manchester City', 'Man United' : 'Manchester United', 'Newcastle' : 'Newcastle United',
                  'Nott\'m Forest': 'Nottingham Forest',  'West Brom' : 'West Bromwich Albion', 'Sheffield Utd' : 'Sheffield United',
                  'Spurs' : 'Tottenham', 'Wolves' : 'Wolverhampton Wanderers'}

odds_df['Home_Team'] = odds_df['Home_Team'].map(replace_values).fillna(odds.HomeTeam)
odds_df['Away_Team'] = odds_df['Away_Team'].map(replace_values).fillna(odds.AwayTeam)
Epl_official_ready['Home_Team'] = Epl_official_ready['Home_Team'].map(replace_values).fillna(Epl_official_ready.Home_Team)
Epl_official_ready['Away_Team'] = Epl_official_ready['Away_Team'].map(replace_values).fillna(Epl_official_ready.Away_Team)

In [67]:
pick_read = open('drive/My Drive/Salka ML football/data/tempdót/Eploffical_season1819.pickle','rb')
Eploffical_season1819 = pickle.load(pick_read)

In [69]:
#breyta í prósentur
for x in ['OddsH','OddsD','OddsA']:
    odds_df[x]=1/odds_df[x]

odds_df.shape

(3040, 15)

In [70]:
replace_values = {'H' : '0', 'D' : '1', 'A' : '2'}
odds_df['Half Time Result'] = odds_df['Half Time Result'].map(replace_values)

In [79]:
odds_df

,Date,Home_Team,Away_Team,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA
0,11/08/2017,Arsenal,Leicester,1,27,6,10,3,9,4,9,12,0.653595,0.222222,0.153846
1,12/08/2017,Brighton,Manchester City,1,6,14,2,4,3,10,6,9,0.090909,0.181818,0.751880
2,12/08/2017,Chelsea,Burnley,2,19,10,6,5,8,5,16,11,0.800000,0.153846,0.066667
3,12/08/2017,Crystal Palace,Huddersfield,2,14,8,4,6,12,9,7,19,0.546448,0.277778,0.200000
4,12/08/2017,Everton,Stoke,0,9,9,4,1,6,7,13,10,0.588235,0.263158,0.173913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,25/05/2025,Newcastle United,Everton,1,17,14,6,6,12,3,12,9,0.751880,0.181818,0.117647
376,25/05/2025,Nottingham Forest,Chelsea,1,10,6,2,2,7,4,10,11,0.333333,0.266667,0.454545
377,25/05/2025,Southampton,Arsenal,2,7,23,2,8,5,8,7,8,0.117647,0.166667,0.769231
378,25/05/2025,Tottenham,Brighton,0,4,23,2,8,2,11,13,8,0.238095,0.230947,0.588235


In [80]:
odds_df['Date'] = pd.to_datetime(odds_df['Date'], format='%d/%m/%Y')
odds_df['Date'] = odds_df['Date'].dt.strftime('%Y/%m/%d')

In [86]:
odds_df

,Date,Home_Team,Away_Team,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA
0,2017/08/11,Arsenal,Leicester,1,27,6,10,3,9,4,9,12,0.653595,0.222222,0.153846
1,2017/08/12,Brighton,Manchester City,1,6,14,2,4,3,10,6,9,0.090909,0.181818,0.751880
2,2017/08/12,Chelsea,Burnley,2,19,10,6,5,8,5,16,11,0.800000,0.153846,0.066667
3,2017/08/12,Crystal Palace,Huddersfield,2,14,8,4,6,12,9,7,19,0.546448,0.277778,0.200000
4,2017/08/12,Everton,Stoke,0,9,9,4,1,6,7,13,10,0.588235,0.263158,0.173913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2025/05/25,Newcastle United,Everton,1,17,14,6,6,12,3,12,9,0.751880,0.181818,0.117647
376,2025/05/25,Nottingham Forest,Chelsea,1,10,6,2,2,7,4,10,11,0.333333,0.266667,0.454545
377,2025/05/25,Southampton,Arsenal,2,7,23,2,8,5,8,7,8,0.117647,0.166667,0.769231
378,2025/05/25,Tottenham,Brighton,0,4,23,2,8,2,11,13,8,0.238095,0.230947,0.588235


In [87]:
#mergea saman

test = pd.merge(left=understat_epl_ready, right=odds_df,
                  left_on= ['Date','Home_Team', 'Away_Team'],
                   right_on=['Date','Home_Team', 'Away_Team'],
                  how='left')




In [91]:
test

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,...,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA
0,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,...,6,10,3,9,4,9,12,0.653595,0.222222,0.153846
1,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,...,14,2,4,3,10,6,9,0.090909,0.181818,0.751880
2,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,...,14,4,5,3,3,14,8,0.166667,0.238095,0.617284
3,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,...,9,6,2,8,2,15,3,0.416667,0.303030,0.303030
4,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,...,4,2,0,13,0,10,13,0.617284,0.250000,0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,2025/05/25,Newcastle United,Everton,0,1,1.1591,1.07279,6,1,6.2500,...,14,6,6,12,3,12,9,0.751880,0.181818,0.117647
3036,2025/05/25,Nottingham Forest,Chelsea,0,1,1.58014,1.30874,3,1,11.3889,...,6,2,2,7,4,10,11,0.333333,0.266667,0.454545
3037,2025/05/25,Fulham,Manchester City,0,2,2.21797,2.45923,5,16,22.1538,...,20,3,5,1,6,11,5,0.200000,0.238095,0.625000
3038,2025/05/25,Manchester United,Aston Villa,2,0,2.96259,0.335222,24,5,5.1071,...,6,10,1,4,3,10,10,0.230947,0.238095,0.588235


In [94]:
Epl_data_ready = pd.merge(left=test, right=Epl_official_ready,
                  left_on= ['Date','Home_Team', 'Away_Team'],
                   right_on=['Date','Home_Team', 'Away_Team'],
                  how='left')

In [95]:
Epl_data_ready

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,...,Home_Clearances,Away_Clearances,Home_Corners,Away_Corners,Home_Offsides,Away_Offsides,Home_Fouls_conceded,Away_Fouls_conceded,Home_label.stat.ontarget_scoring_att,Away_label.stat.ontarget_scoring_att
0,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,...,30.0,24.0,9.0,4.0,5.0,3.0,9.0,12.0,0.0,0.0
1,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,...,39.0,8.0,3.0,10.0,6.0,1.0,6.0,9.0,0.0,0.0
2,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,...,28.0,22.0,3.0,3.0,3.0,1.0,14.0,8.0,0.0,0.0
3,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,...,26.0,17.0,8.0,2.0,2.0,0.0,15.0,3.0,0.0,0.0
4,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,...,11.0,38.0,13.0,0.0,0.0,1.0,10.0,13.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3377,2025/05/25,Newcastle United,Everton,0,1,1.1591,1.07279,6,1,6.2500,...,21.0,25.0,12.0,3.0,0.0,0.0,12.0,9.0,6.0,6.0
3378,2025/05/25,Nottingham Forest,Chelsea,0,1,1.58014,1.30874,3,1,11.3889,...,22.0,36.0,7.0,4.0,5.0,1.0,10.0,11.0,2.0,2.0
3379,2025/05/25,Fulham,Manchester City,0,2,2.21797,2.45923,5,16,22.1538,...,29.0,14.0,1.0,6.0,2.0,0.0,11.0,5.0,3.0,5.0
3380,2025/05/25,Manchester United,Aston Villa,2,0,2.96259,0.335222,24,5,5.1071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
#nú má framkvæma feature extration og reikna út lýsandi gögn fyrir ml líkönin
#byrjum á að droppa dálkum sem lýsa því sama sem eru ekki nothæfir
#gaman væri að setja inn einhverskonar rating system svipað elo

In [110]:
pd.set_option('display.max_columns', None)

In [104]:
Epl_data_ready.head(2)

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA,Home_Possession_%,Away_Possession_%,Home_Shots_on_target,Away_Shots_on_target,Home_Shots,Away_Shots,Home_Touches,Away_Touches,Home_Passes,Away_Passes,Home_Tackles,Away_Tackles,Home_Clearances,Away_Clearances,Home_Corners,Away_Corners,Home_Offsides,Away_Offsides,Home_Fouls_conceded,Away_Fouls_conceded,Home_label.stat.ontarget_scoring_att,Away_label.stat.ontarget_scoring_att,target
0,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,13.5455,1,27,6,10,3,9,4,9,12,0.653595,0.222222,0.153846,70.0,30.0,10.0,3.0,27.0,6.0,859.0,459.0,632.0,263.0,23.0,17.0,30.0,24.0,9.0,4.0,5.0,3.0,9.0,12.0,0.0,0.0,W
1,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,5.4444,1,6,14,2,4,3,10,6,9,0.090909,0.181818,0.751880,21.8,78.2,2.0,4.0,6.0,14.0,375.0,918.0,213.0,768.0,10.0,10.0,39.0,8.0,3.0,10.0,6.0,1.0,6.0,9.0,0.0,0.0,W


In [106]:
Epl_data_ready = Epl_data_ready.drop(columns=['HomeShots', 'AwayShots','HomeShotOT', 'AwayShotsOT',
                                              'HomeCorners', 'AwayCorners', 'HomeFouls', 'AwayFouls',
                                              'Home_label.stat.ontarget_scoring_att', 'Away_label.stat.ontarget_scoring_att'])

In [107]:
#hvaða lið vann? Heima, draw eða útilið, aðeins öðruvísi en svipað fall að ofan.
def match_winner(winner):
  winner['target'] = 'W'
  winner['target'][winner['Home_Goals']==winner['Away_Goals']]='D'
  winner['target'][winner['Home_Goals']<winner['Away_Goals']]='L'

  return winner



In [117]:
Epl_data_ready = match_winner(Epl_data_ready)

In [118]:
Epl_data_ready.head()

,Date,Home_Team,Away_Team,Home_Goals,Away_Goals,Home_xG,Away_xG,Home_Deep,Away_Deep,Home_ppda,Away_ppda,Half Time Result,OddsH,OddsD,OddsA,Home_Possession_%,Away_Possession_%,Home_Shots_on_target,Away_Shots_on_target,Home_Shots,Away_Shots,Home_Touches,Away_Touches,Home_Passes,Away_Passes,Home_Tackles,Away_Tackles,Home_Clearances,Away_Clearances,Home_Corners,Away_Corners,Home_Offsides,Away_Offsides,Home_Fouls_conceded,Away_Fouls_conceded,target
0,2017/08/11,Arsenal,Leicester,4,3,2.54329,1.46495,13,2,5.4444,13.5455,1,0.653595,0.222222,0.153846,70.0,30.0,10.0,3.0,27.0,6.0,859.0,459.0,632.0,263.0,23.0,17.0,30.0,24.0,9.0,4.0,5.0,3.0,9.0,12.0,W
1,2017/08/12,Brighton,Manchester City,0,2,0.276343,1.86751,2,18,67.3333,5.4444,1,0.090909,0.181818,0.751880,21.8,78.2,2.0,4.0,6.0,14.0,375.0,918.0,213.0,768.0,10.0,10.0,39.0,8.0,3.0,10.0,6.0,1.0,6.0,9.0,L
2,2017/08/12,Watford,Liverpool,3,3,2.17647,2.61549,2,10,11.2609,14.7143,0,0.166667,0.238095,0.617284,45.6,54.4,4.0,5.0,9.0,14.0,595.0,682.0,395.0,477.0,19.0,20.0,28.0,22.0,3.0,3.0,3.0,1.0,14.0,8.0,D
3,2017/08/12,West Bromwich Albion,Bournemouth,1,0,1.18399,0.378659,6,6,17.3889,12.0000,0,0.416667,0.303030,0.303030,28.7,71.3,6.0,2.0,16.0,9.0,419.0,775.0,242.0,612.0,19.0,16.0,26.0,17.0,8.0,2.0,2.0,0.0,15.0,3.0,W
4,2017/08/12,Southampton,Swansea,0,0,2.21748,0.406196,16,1,8.5200,12.8824,1,0.617284,0.250000,0.153846,59.6,40.4,2.0,0.0,29.0,4.0,707.0,532.0,518.0,365.0,12.0,11.0,11.0,38.0,13.0,0.0,0.0,1.0,10.0,13.0,D


In [ ]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

def get_cumulative_points(matchres, GW):
    matchres_points = matchres.applymap(get_points)
    for i in range(2, GW+1):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]

    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchres_points

def get_match_result(stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    # build dict where value is list of match results
    for i in range(len(stat)):
        if stat.iloc[i].target == 'W':
            teams[stat.iloc[i].HomeTeam].append('W')
            teams[stat.iloc[i].AwayTeam].append('L')
        elif stat.iloc[i].target == 'L':
            teams[stat.iloc[i].AwayTeam].append('W')
            teams[stat.iloc[i].HomeTeam].append('L')
        else:
            teams[stat.iloc[i].AwayTeam].append('D')
            teams[stat.iloc[i].HomeTeam].append('D')

    return pd.DataFrame(data=teams, index = [i for i in range(1, max(stat['GW']+1))]).T

def get_agg_points(stat):
    matchres = get_match_result(stat)
    cum_pts = get_cumulative_points(matchres, max(stat['GW']))
    HTP = []
    ATP = []
    j = 0
    for i in range(len(stat)):
        ht = stat.iloc[i].HomeTeam
        at = stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j += 1

    stat['HTP'] = HTP
    stat['ATP'] = ATP
    return stat


In [ ]:
df_2016 = get_agg_points(df_2016)
df_2017 = get_agg_points(df_2017)
df_2018 = get_agg_points(df_2018)
df_2019 = get_agg_points(df_2019)
df_2020 = get_agg_points(df_2020)
df_2021 = get_agg_points(df_2021)
df_2022 = get_agg_points(df_2022)

In [ ]:
df_2022

,Unnamed: 0,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,...,ATGC,HTxGS,ATxGS,HTxGC,ATxGC,winner,glicko_diff,target,HTP,ATP
0,0,2022-08-05 19:00:00,Crystal Palace,Arsenal,0,2,1.206370,1.436010,1,0,...,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0
1,1,2022-08-06 11:30:00,Fulham,Liverpool,2,2,1.268220,2.341110,1,0,...,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0
2,2,2022-08-06 14:00:00,Bournemouth,Aston Villa,2,0,0.588341,0.488895,1,0,...,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,W,0,0
3,3,2022-08-06 14:00:00,Leeds,Wolverhampton Wanderers,2,1,0.889170,1.101190,1,0,...,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,W,0,0
4,4,2022-08-06 14:00:00,Newcastle United,Nottingham Forest,2,0,1.859100,0.235825,1,0,...,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,W,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,2023-05-28 15:30:00,Everton,Bournemouth,1,0,1.129330,0.432084,38,33,...,70,47.520506,39.651881,68.150778,66.472764,0,-324.743046,W,33,39
376,376,2023-05-28 15:30:00,Leeds,Tottenham,1,4,1.191510,2.033150,38,47,...,62,46.604455,55.800640,67.547883,50.469218,2,-585.675144,L,31,57
377,377,2023-05-28 15:30:00,Leicester,West Ham,2,1,2.030130,0.970352,38,49,...,53,46.956869,50.240380,58.818999,53.282043,0,-33.584483,W,31,40
378,378,2023-05-28 15:30:00,Manchester United,Fulham,2,1,2.747660,1.881670,38,56,...,51,69.153518,46.103307,48.405969,68.743151,0,73.719011,W,72,52


In [ ]:
#form hjá liðum...
def get_match_result(stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    # build dict where value is list of match results
    for i in range(len(stat)):
        if stat.iloc[i].target == 'W':
            teams[stat.iloc[i].HomeTeam].append('W')
            teams[stat.iloc[i].AwayTeam].append('L')
        elif stat.iloc[i].target == 'L':
            teams[stat.iloc[i].AwayTeam].append('W')
            teams[stat.iloc[i].HomeTeam].append('L')
        else:
            teams[stat.iloc[i].AwayTeam].append('D')
            teams[stat.iloc[i].HomeTeam].append('D')

    return pd.DataFrame(data=teams, index = [i for i in range(1, max(stat['GW']+1))]).T

def get_form(playing_stat,num):
    form = get_match_result(playing_stat)
    form_final = form.copy()
    for i in range(num, max(playing_stat['GW'])+1):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1
    return form_final

def add_form(playing_stat,num):
    form = get_form(playing_stat,num)
    h = ['-' for i in range(num * 10)]  # since form is not available for n -W (n*10)
    a = ['-' for i in range(num * 10)]

    j = num
    for i in range((num*10),playing_stat.shape[0]):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam

        past = form.loc[ht][j]  # get past n results
        h.append(past[num-1])   # 0 index is most recent

        past = form.loc[at][j]  # get past n results.
        a.append(past[num-1])   # 0 index is most recent

        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HM' + str(num)] = h
    playing_stat['AM' + str(num)] = a


    return playing_stat


def add_form_df(stats):
    stats = add_form(stats,1)
    stats = add_form(stats,2)
    stats = add_form(stats,3)
    stats = add_form(stats,4)
    stats = add_form(stats,5)

    return stats

In [ ]:
df_2016 = add_form_df(df_2016)
df_2017 = add_form_df(df_2017)
df_2018 = add_form_df(df_2018)
df_2019 = add_form_df(df_2019)
df_2020 = add_form_df(df_2020)
df_2021 = add_form_df(df_2021)
df_2022 = add_form_df(df_2022)

In [ ]:
df_2021

,Unnamed: 0,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,...,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5
0,0,2021-08-13 19:00:00,Brentford,Arsenal,2,0,1.888180,1.023850,1,0,...,-,-,-,-,-,-,-,-,-,-
1,1,2021-08-14 11:30:00,Manchester United,Leeds,5,1,1.642480,0.568342,1,0,...,-,-,-,-,-,-,-,-,-,-
2,2,2021-08-14 14:00:00,Burnley,Brighton,1,2,1.795480,1.685300,1,0,...,-,-,-,-,-,-,-,-,-,-
3,3,2021-08-14 14:00:00,Chelsea,Crystal Palace,3,0,1.187090,0.321701,1,0,...,-,-,-,-,-,-,-,-,-,-
4,4,2021-08-14 14:00:00,Everton,Southampton,3,1,2.388630,0.580601,1,0,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,2022-05-22 15:00:00,Crystal Palace,Manchester United,1,0,0.354618,0.782005,38,49,...,L,L,D,W,W,D,W,L,D,L
376,376,2022-05-22 15:00:00,Leicester,Southampton,4,1,3.420580,1.051410,38,58,...,D,L,W,L,W,L,L,D,L,L
377,377,2022-05-22 15:00:00,Liverpool,Wolverhampton Wanderers,3,1,3.193850,1.117080,38,91,...,W,D,W,L,D,D,W,L,W,L
378,378,2022-05-22 15:00:00,Manchester City,Aston Villa,3,2,3.320900,0.247841,38,96,...,D,D,W,D,W,L,W,W,W,W


In [ ]:
merged_df_2016 = pd.merge(left=df_2016, right=odds_2016,
                  left_on= ['HomeTeam', 'AwayTeam'], right_on=['Home_Team', 'Away_Team'],
                  how='left')

In [ ]:
merged_df_2017 = pd.merge(left=df_2017, right=odds_2017,
                  left_on= ['HomeTeam', 'AwayTeam'], right_on=['Home_Team', 'Away_Team'],
                  how='left')

In [ ]:
merged_df_2018 = pd.merge(left=df_2018, right=odds_2018,
                  left_on= ['HomeTeam', 'AwayTeam'], right_on=['Home_Team', 'Away_Team'],
                  how='left')

In [ ]:
merged_df_2019 = pd.merge(left=df_2019, right=odds_2019,
                  left_on= ['HomeTeam', 'AwayTeam'], right_on=['Home_Team', 'Away_Team'],
                  how='left')

In [ ]:
merged_df_2020 = pd.merge(left=df_2020, right=odds_2020,
                  left_on= ['HomeTeam', 'AwayTeam'], right_on=['Home_Team', 'Away_Team'],
                  how='left')

In [ ]:
merged_df_2021 = pd.merge(left=df_2021, right=odds_2021,
                  left_on= ['HomeTeam', 'AwayTeam'], right_on=['Home_Team', 'Away_Team'],
                  how='left')

In [ ]:
merged_df_2022 = pd.merge(left=df_2022, right=odds_2022,
                  left_on= ['HomeTeam', 'AwayTeam'], right_on=['Home_Team', 'Away_Team'],
                  how='left')

In [ ]:
merged_df_2022

,Unnamed: 0,Date_x,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,...,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA
0,0,2022-08-05 19:00:00,Crystal Palace,Arsenal,0,2,1.206370,1.436010,1,0,...,10,2,2,3,5,16,11,0.238095,0.277778,0.540541
1,1,2022-08-06 11:30:00,Fulham,Liverpool,2,2,1.268220,2.341110,1,0,...,11,3,4,4,4,7,9,0.090909,0.166667,0.800000
2,2,2022-08-06 14:00:00,Bournemouth,Aston Villa,2,0,0.588341,0.488895,1,0,...,15,3,2,5,5,18,16,0.266667,0.285714,0.500000
3,3,2022-08-06 14:00:00,Leeds,Wolverhampton Wanderers,2,1,0.889170,1.101190,1,0,...,15,4,6,6,4,13,9,0.444444,0.294118,0.312500
4,4,2022-08-06 14:00:00,Newcastle United,Nottingham Forest,2,0,1.859100,0.235825,1,0,...,5,10,0,11,1,9,14,0.602410,0.263158,0.190476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,2023-05-28 15:30:00,Everton,Bournemouth,1,0,1.129330,0.432084,38,33,...,7,6,2,9,3,11,12,0.666667,0.222222,0.153846
376,376,2023-05-28 15:30:00,Leeds,Tottenham,1,4,1.191510,2.033150,38,47,...,11,2,7,12,3,7,5,0.363636,0.263158,0.434783
377,377,2023-05-28 15:30:00,Leicester,West Ham,2,1,2.030130,0.970352,38,49,...,16,4,3,3,5,8,10,0.523560,0.263158,0.277778
378,378,2023-05-28 15:30:00,Manchester United,Fulham,2,1,2.747660,1.881670,38,56,...,10,8,3,5,4,14,10,0.666667,0.230947,0.166667


In [ ]:
odds_2021

,Date,Home_Team,Away_Team,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA
0,13/08/2021,Brentford,Arsenal,0,8,22,3,4,2,5,12,8,0.250000,0.294118,0.512821
1,14/08/2021,Manchester United,Leeds,0,16,10,8,3,5,4,11,9,0.653595,0.222222,0.173913
2,14/08/2021,Burnley,Brighton,0,14,14,3,8,7,6,10,7,0.322581,0.322581,0.408163
3,14/08/2021,Chelsea,Crystal Palace,0,13,4,6,1,5,2,15,11,0.800000,0.173913,0.076923
4,14/08/2021,Everton,Southampton,2,14,6,6,3,6,8,13,15,0.526316,0.285714,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,22/05/2022,Crystal Palace,Manchester United,0,6,10,3,4,3,6,12,22,0.333333,0.277778,0.444444
376,22/05/2022,Leicester,Southampton,1,12,7,6,2,3,3,10,5,0.540541,0.250000,0.263158
377,22/05/2022,Liverpool,Wolverhampton Wanderers,1,29,7,8,5,5,3,6,3,0.877193,0.117647,0.066667
378,22/05/2022,Manchester City,Aston Villa,2,24,4,5,2,13,1,5,11,0.862069,0.125000,0.058824


In [ ]:
#merge glicko hér að ofan við þennan df, þarf svo að laga aðeins cols.

In [ ]:
df_final = pd.concat([
                      merged_df_2016,
                      merged_df_2017,
                      merged_df_2018,
                      merged_df_2019,
                      merged_df_2020,
                      merged_df_2021,
                      merged_df_2022], axis=0)

In [ ]:
df_final

,Unnamed: 0,Date_x,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,...,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA
0,0,2016-08-13 15:30:00,Hull,Leicester,2,1,0.740018,2.456310,1,0,...,18,5,5,5,3,8,17,0.222222,0.277778,0.523560
1,1,2016-08-13 18:00:00,Crystal Palace,West Bromwich Albion,0,1,0.990062,0.684593,1,0,...,13,4,3,3,6,12,15,0.500000,0.303030,0.222222
2,2,2016-08-13 18:00:00,Everton,Tottenham,1,1,0.790063,1.123480,1,0,...,13,6,4,5,6,10,14,0.312500,0.294118,0.416667
3,3,2016-08-13 18:00:00,Burnley,Swansea,0,1,1.049030,1.720890,1,0,...,17,3,9,7,4,10,14,0.416667,0.303030,0.307692
4,4,2016-08-13 18:00:00,Middlesbrough,Stoke,1,1,1.805230,0.485626,1,0,...,12,2,1,9,6,18,14,0.420168,0.312500,0.294118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,2023-05-28 15:30:00,Everton,Bournemouth,1,0,1.129330,0.432084,38,33,...,7,6,2,9,3,11,12,0.666667,0.222222,0.153846
376,376,2023-05-28 15:30:00,Leeds,Tottenham,1,4,1.191510,2.033150,38,47,...,11,2,7,12,3,7,5,0.363636,0.263158,0.434783
377,377,2023-05-28 15:30:00,Leicester,West Ham,2,1,2.030130,0.970352,38,49,...,16,4,3,3,5,8,10,0.523560,0.263158,0.277778
378,378,2023-05-28 15:30:00,Manchester United,Fulham,2,1,2.747660,1.881670,38,56,...,10,8,3,5,4,14,10,0.666667,0.230947,0.166667


In [ ]:
# Gets the form points.
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0


def get_form_points(string):
    total = 0
    for letter in string:
        total += get_points(letter)
    return total

df_final['HTFormPtsStr'] = df_final['HM1'] + df_final['HM2'] + df_final['HM3'] + df_final['HM4'] + df_final['HM5']
df_final['ATFormPtsStr'] = df_final['AM1'] + df_final['AM2'] + df_final['AM3'] + df_final['AM4'] + df_final['AM5']

df_final['HTFormPts'] = df_final['HTFormPtsStr'].apply(get_form_points)
df_final['ATFormPts'] = df_final['ATFormPtsStr'].apply(get_form_points)

In [ ]:
pd.set_option('display.max_columns', None)
df_final

,Unnamed: 0,Date_x,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,ATGS,HTGC,ATGC,HTxGS,ATxGS,HTxGC,ATxGC,winner,glicko_diff,target,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,Date_y,Home_Team,Away_Team,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts
0,0,2016-08-13 15:30:00,Hull,Leicester,2,1,0.740018,2.456310,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,W,0,0,-,-,-,-,-,-,-,-,-,-,13/08/16,Hull,Leicester,0,14,18,5,5,5,3,8,17,0.222222,0.277778,0.523560,-----,-----,0,0
1,1,2016-08-13 18:00:00,Crystal Palace,West Bromwich Albion,0,1,0.990062,0.684593,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,-,-,-,-,-,-,-,-,-,-,13/08/16,Crystal Palace,West Bromwich Albion,1,14,13,4,3,3,6,12,15,0.500000,0.303030,0.222222,-----,-----,0,0
2,2,2016-08-13 18:00:00,Everton,Tottenham,1,1,0.790063,1.123480,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,-,-,-,-,-,-,-,-,-,-,13/08/16,Everton,Tottenham,0,12,13,6,4,5,6,10,14,0.312500,0.294118,0.416667,-----,-----,0,0
3,3,2016-08-13 18:00:00,Burnley,Swansea,0,1,1.049030,1.720890,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,-,-,-,-,-,-,-,-,-,-,13/08/16,Burnley,Swansea,1,10,17,3,9,7,4,10,14,0.416667,0.303030,0.307692,-----,-----,0,0
4,4,2016-08-13 18:00:00,Middlesbrough,Stoke,1,1,1.805230,0.485626,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,-,-,-,-,-,-,-,-,-,-,13/08/16,Middlesbrough,Stoke,0,12,12,2,1,9,6,18,14,0.420168,0.312500,0.294118,-----,-----,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,2023-05-28 15:30:00,Everton,Bournemouth,1,0,1.129330,0.432084,38,33,37,57,70,47.520506,39.651881,68.150778,66.472764,0,-324.743046,W,33,39,D,L,L,L,W,L,D,W,L,W,28/05/2023,Everton,Bournemouth,1,13,7,6,2,9,3,11,12,0.666667,0.222222,0.153846,DLWDL,LLLWW,5,6
376,376,2023-05-28 15:30:00,Leeds,Tottenham,1,4,1.191510,2.033150,38,47,66,74,62,46.604455,55.800640,67.547883,50.469218,2,-585.675144,L,31,57,L,L,D,L,L,W,L,L,D,D,28/05/2023,Leeds,Tottenham,2,19,11,2,7,12,3,7,5,0.363636,0.263158,0.434783,LDLLD,LLWLD,2,4
377,377,2023-05-28 15:30:00,Leicester,West Ham,2,1,2.030130,0.970352,38,49,41,67,53,46.956869,50.240380,58.818999,53.282043,0,-33.584483,W,31,40,D,W,L,L,L,W,D,L,D,L,28/05/2023,Leicester,West Ham,0,13,16,4,3,3,5,8,10,0.523560,0.263158,0.277778,DLLDD,WLWLL,3,6
378,378,2023-05-28 15:30:00,Manchester United,Fulham,2,1,2.747660,1.881670,38,56,54,42,51,69.153518,46.103307,48.405969,68.743151,0,73.719011,W,72,52,W,D,W,W,W,W,L,L,L,L,28/05/2023,Manchester United,Fulham,1,21,10,8,3,5,4,14,10,0.666667,0.230947,0.166667,WWWLL,DWWLL,9,7


In [ ]:
#form dugar, þarf ekki mörk skoruð yfir tímabil né stigasöfnun, má svosem halda því en þá væri ágætt að skoða aðeins kóðann
#og bæta við xG söfnun.

In [ ]:
df_final = df_final.drop(['Unnamed: 0', 'Date_x', 'Home_Team', 'Away_Team', 'HM1',
                          'AM1', 'HM2', 'AM2', 'HM3', 'AM3', 'HM4', 'AM4',
                          'HM5', 'AM5'], axis=1)

In [ ]:
df_final = df_final.rename({'Date_y':'Date'}, axis=1)

In [ ]:
#gameid
df_final.insert(0, 'GameId', range(1, len(df_final)+1))

In [ ]:
df_final

,GameId,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,ATGS,HTGC,ATGC,HTxGS,ATxGS,HTxGC,ATxGC,winner,glicko_diff,target,HTP,ATP,Date,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts
0,1,Hull,Leicester,2,1,0.740018,2.456310,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,W,0,0,13/08/16,0,14,18,5,5,5,3,8,17,0.222222,0.277778,0.523560,-----,-----,0,0
1,2,Crystal Palace,West Bromwich Albion,0,1,0.990062,0.684593,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,13/08/16,1,14,13,4,3,3,6,12,15,0.500000,0.303030,0.222222,-----,-----,0,0
2,3,Everton,Tottenham,1,1,0.790063,1.123480,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,13/08/16,0,12,13,6,4,5,6,10,14,0.312500,0.294118,0.416667,-----,-----,0,0
3,4,Burnley,Swansea,0,1,1.049030,1.720890,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,13/08/16,1,10,17,3,9,7,4,10,14,0.416667,0.303030,0.307692,-----,-----,0,0
4,5,Middlesbrough,Stoke,1,1,1.805230,0.485626,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,13/08/16,0,12,12,2,1,9,6,18,14,0.420168,0.312500,0.294118,-----,-----,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2656,Everton,Bournemouth,1,0,1.129330,0.432084,38,33,37,57,70,47.520506,39.651881,68.150778,66.472764,0,-324.743046,W,33,39,28/05/2023,1,13,7,6,2,9,3,11,12,0.666667,0.222222,0.153846,DLWDL,LLLWW,5,6
376,2657,Leeds,Tottenham,1,4,1.191510,2.033150,38,47,66,74,62,46.604455,55.800640,67.547883,50.469218,2,-585.675144,L,31,57,28/05/2023,2,19,11,2,7,12,3,7,5,0.363636,0.263158,0.434783,LDLLD,LLWLD,2,4
377,2658,Leicester,West Ham,2,1,2.030130,0.970352,38,49,41,67,53,46.956869,50.240380,58.818999,53.282043,0,-33.584483,W,31,40,28/05/2023,0,13,16,4,3,3,5,8,10,0.523560,0.263158,0.277778,DLLDD,WLWLL,3,6
378,2659,Manchester United,Fulham,2,1,2.747660,1.881670,38,56,54,42,51,69.153518,46.103307,48.405969,68.743151,0,73.719011,W,72,52,28/05/2023,1,21,10,8,3,5,4,14,10,0.666667,0.230947,0.166667,WWWLL,DWWLL,9,7


Hér vil ég gera smá breytingar á framsetningu. Laga til breytur með því að setja fram "rolling average" eða exponentially weighted gildi á þeim. Form breyta, hversu vel er lið að standa sig í seinustu 5 eða 10 leikjum. Þarf svo að færa til staðsetningu breytanna þar sem módelið má ekki spá fyrir(y) um leik x með tölfræði úr leik x. Get notað shift functionið í pandas eftir að ég hef búið til breyturnar. Einnig væri áhugavert að setja fram einhverskonar breytu sem mælir styrk liðs varnar og sóknarlega.

In [ ]:
cols = ['GameId','AwayTeam','AwayGoals', 'AwayxG', 'AwayShots', 'AwayShotsOT', 'AwayCorners', 'AwayFouls']
#Away = df_final(columns = cols)
Away = df_final.filter(cols, axis=1)

In [ ]:
cols = ['GameId','HomeTeam','HomeGoals', 'HomexG', 'HomeShots', 'HomeShotOT', 'HomeCorners', 'HomeFouls']
#Home = df_final(columns = cols)
Home = df_final.filter(cols, axis=1)

In [ ]:
Home.columns = ['GameId', 'Team', 'Goals', 'xG', 'Shots', 'ShotsOT', 'Corners', 'Fouls']

Away.columns = ['GameId', 'Team', 'Goals', 'xG', 'Shots', 'ShotsOT', 'Corners', 'Fouls']

In [ ]:
stat_df = pd.concat([Home,Away]).sort_values(by='GameId').reset_index(drop=True)
stat_df

,GameId,Team,Goals,xG,Shots,ShotsOT,Corners,Fouls
0,1,Hull,2,0.740018,14,5,8,5
1,1,Leicester,1,2.456310,18,5,17,3
2,2,West Bromwich Albion,1,0.684593,13,3,15,6
3,2,Crystal Palace,0,0.990062,14,4,12,3
4,3,Everton,1,0.790063,12,6,10,5
...,...,...,...,...,...,...,...,...
5315,2658,Leicester,2,2.030130,13,4,8,3
5316,2659,Fulham,1,1.881670,10,3,10,4
5317,2659,Manchester United,2,2.747660,21,8,14,5
5318,2660,Southampton,4,1.339640,15,10,4,2


In [ ]:
stat_cols = ['Goals','xG', 'Shots', 'ShotsOT', 'Corners', 'Fouls']

for s in stat_cols:
    stat_df[s+'_ema'] = stat_df.groupby('Team')[s].transform(lambda x:x.ewm(span=38).mean()) # reyna að ná tímabilsstyrk

stat_df.tail()

,GameId,Team,Goals,xG,Shots,ShotsOT,Corners,Fouls,Goals_ema,xG_ema,Shots_ema,ShotsOT_ema,Corners_ema,Fouls_ema
5315,2658,Leicester,2,2.03013,13,4,8,3,1.325073,1.421728,11.018464,4.115467,10.825284,3.652417
5316,2659,Fulham,1,1.88167,10,3,10,4,1.336740,1.197659,11.032152,3.707146,11.314043,4.875983
5317,2659,Manchester United,2,2.74766,21,8,14,5,1.519620,2.001323,16.316307,5.851539,10.896578,5.546445
5318,2660,Southampton,4,1.33964,15,10,4,2,1.089496,1.069648,10.829354,3.878071,10.945773,4.323809
5319,2660,Liverpool,4,4.00345,30,8,10,9,2.111919,2.167772,16.205742,5.508938,11.300264,6.215982


In [ ]:
stat_cols = ['Goals','xG', 'Shots', 'ShotsOT', 'Corners', 'Fouls']

for s in stat_cols:
    stat_df[s+'_5ma'] = stat_df.groupby('Team')[s].transform(lambda x:x.rolling(window=5, min_periods=1).mean())

stat_df.tail()

,GameId,Team,Goals,xG,Shots,ShotsOT,Corners,Fouls,Goals_ema,xG_ema,Shots_ema,ShotsOT_ema,Corners_ema,Fouls_ema,Goals_5ma,xG_5ma,Shots_5ma,ShotsOT_5ma,Corners_5ma,Fouls_5ma
5315,2658,Leicester,2,2.03013,13,4,8,3,1.325073,1.421728,11.018464,4.115467,10.825284,3.652417,1.4,1.778411,10.2,4.8,9.8,3.4
5316,2659,Fulham,1,1.88167,10,3,10,4,1.336740,1.197659,11.032152,3.707146,11.314043,4.875983,2.0,1.520119,11.2,4.2,11.2,5.2
5317,2659,Manchester United,2,2.74766,21,8,14,5,1.519620,2.001323,16.316307,5.851539,10.896578,5.546445,1.8,2.736636,21.0,7.0,10.2,6.2
5318,2660,Southampton,4,1.33964,15,10,4,2,1.089496,1.069648,10.829354,3.878071,10.945773,4.323809,1.8,1.312217,9.6,4.0,10.0,3.8
5319,2660,Liverpool,4,4.00345,30,8,10,9,2.111919,2.167772,16.205742,5.508938,11.300264,6.215982,2.0,2.071706,17.2,5.2,14.4,7.2


In [ ]:
stat_cols = ['Goals_ema','xG_ema','Shots_ema', 'ShotsOT_ema', 'Corners_ema', 'Fouls_ema',
             'Goals_5ma','xG_5ma','Shots_5ma', 'ShotsOT_5ma', 'Corners_5ma', 'Fouls_5ma']
stat_df = stat_df.sort_values(by='GameId').reset_index(drop=True)
for s in stat_cols:
    stat_df[s] = stat_df.groupby('Team')[s].shift(1)

In [ ]:
#þetta veldur nokkum nan breytum, bæði fyrstu leikirnir sökum shift og svo koma upp nan þegar að nýliðar komast upp í deild þeirra bestu
#annað hvort væri hægt að gefa þeim eitthvað global mean eða bara droppa þeim röðum, ath að þegar
# spáð er þarf að gefa þeim gildi fyrst...
#spurning er hversu mikið af þessu ég þarf.

In [ ]:
stat_df

,GameId,Team,Goals,xG,Shots,ShotsOT,Corners,Fouls,Goals_ema,xG_ema,Shots_ema,ShotsOT_ema,Corners_ema,Fouls_ema,Goals_5ma,xG_5ma,Shots_5ma,ShotsOT_5ma,Corners_5ma,Fouls_5ma
0,1,Hull,2,0.740018,14,5,8,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Leicester,1,2.456310,18,5,17,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,West Bromwich Albion,1,0.684593,13,3,15,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Crystal Palace,0,0.990062,14,4,12,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,Everton,1,0.790063,12,6,10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,2658,Leicester,2,2.030130,13,4,8,3,1.288591,1.388841,10.911354,4.121708,10.978002,3.687683,1.2,1.789573,10.6,4.6,11.2,4.0
5316,2659,Fulham,1,1.881670,10,3,10,4,1.354989,1.160589,11.088089,3.745470,11.385258,4.923457,2.0,1.230148,10.0,3.8,11.0,4.8
5317,2659,Manchester United,2,2.747660,21,8,14,5,1.493654,1.960980,16.063134,5.735406,10.728826,5.575982,1.4,2.451314,20.0,6.4,10.2,6.4
5318,2660,Southampton,4,1.339640,15,10,4,2,0.932171,1.055053,10.603914,3.547156,11.321221,4.449420,1.0,1.139695,8.8,2.4,11.0,4.6


In [ ]:
stat_df = pd.concat(
    [
        stat_df.iloc[::2].reset_index(drop=True),
        stat_df.iloc[1::2].reset_index(drop=True)
    ], axis=1
)
stat_df

,GameId,Team,Goals,xG,Shots,ShotsOT,Corners,Fouls,Goals_ema,xG_ema,Shots_ema,ShotsOT_ema,Corners_ema,Fouls_ema,Goals_5ma,xG_5ma,Shots_5ma,ShotsOT_5ma,Corners_5ma,Fouls_5ma,GameId,Team,Goals,xG,Shots,ShotsOT,Corners,Fouls,Goals_ema,xG_ema,Shots_ema,ShotsOT_ema,Corners_ema,Fouls_ema,Goals_5ma,xG_5ma,Shots_5ma,ShotsOT_5ma,Corners_5ma,Fouls_5ma
0,1,Hull,2,0.740018,14,5,8,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Leicester,1,2.456310,18,5,17,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,West Bromwich Albion,1,0.684593,13,3,15,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Crystal Palace,0,0.990062,14,4,12,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Everton,1,0.790063,12,6,10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Tottenham,1,1.123480,13,4,14,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Burnley,0,1.049030,10,3,10,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Swansea,1,1.720890,17,9,14,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Middlesbrough,1,1.805230,12,2,18,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Stoke,1,0.485626,12,1,14,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2656,Bournemouth,0,0.432084,7,2,12,3,1.044739,1.187594,10.191589,3.685048,10.069962,4.063956,1.2,1.160216,10.6,3.4,10.0,3.6,2656,Everton,1,1.129330,13,6,11,9,1.032325,1.409574,11.856460,4.111920,10.679454,4.446944,1.8,2.132290,14.4,5.0,11.0,5.6
2656,2657,Tottenham,4,2.033150,11,7,5,3,1.742311,1.526763,13.475907,5.056770,11.409294,5.118106,1.6,1.556006,12.6,5.4,10.8,5.4,2657,Leeds,1,1.191510,19,2,7,12,1.233565,1.233433,11.759241,3.837085,12.175745,4.792627,1.2,1.247156,10.6,3.6,14.2,3.4
2657,2658,West Ham,1,0.970352,16,3,10,5,1.308893,1.395155,11.728200,3.997522,9.044661,5.467341,1.4,1.384839,10.4,4.6,8.0,5.4,2658,Leicester,2,2.030130,13,4,8,3,1.288591,1.388841,10.911354,4.121708,10.978002,3.687683,1.2,1.789573,10.6,4.6,11.2,4.0
2658,2659,Fulham,1,1.881670,10,3,10,4,1.354989,1.160589,11.088089,3.745470,11.385258,4.923457,2.0,1.230148,10.0,3.8,11.0,4.8,2659,Manchester United,2,2.747660,21,8,14,5,1.493654,1.960980,16.063134,5.735406,10.728826,5.575982,1.4,2.451314,20.0,6.4,10.2,6.4


In [ ]:
#til að laga cols
stat_df.to_csv('drive/My Drive/stat_df.csv')
stat_df = pd.read_csv('drive/My Drive/stat_df.csv')

In [ ]:
#nokkrir hlutir sem þarf að skoða, nan í fyrstu leikjum nýliða en samt á vitlausum stað?, skoða is nan fjöldinn
stat_df

,Unnamed: 0,GameId,Team,Goals,xG,Shots,ShotsOT,Corners,Fouls,Goals_ema,xG_ema,Shots_ema,ShotsOT_ema,Corners_ema,Fouls_ema,Goals_5ma,xG_5ma,Shots_5ma,ShotsOT_5ma,Corners_5ma,Fouls_5ma,GameId.1,Team.1,Goals.1,xG.1,Shots.1,ShotsOT.1,Corners.1,Fouls.1,Goals_ema.1,xG_ema.1,Shots_ema.1,ShotsOT_ema.1,Corners_ema.1,Fouls_ema.1,Goals_5ma.1,xG_5ma.1,Shots_5ma.1,ShotsOT_5ma.1,Corners_5ma.1,Fouls_5ma.1
0,0,1,Hull,2,0.740018,14,5,8,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Leicester,1,2.456310,18,5,17,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,West Bromwich Albion,1,0.684593,13,3,15,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Crystal Palace,0,0.990062,14,4,12,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,Everton,1,0.790063,12,6,10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Tottenham,1,1.123480,13,4,14,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,Burnley,0,1.049030,10,3,10,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Swansea,1,1.720890,17,9,14,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,Middlesbrough,1,1.805230,12,2,18,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Stoke,1,0.485626,12,1,14,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2655,2656,Bournemouth,0,0.432084,7,2,12,3,1.044739,1.187594,10.191589,3.685048,10.069962,4.063956,1.2,1.160216,10.6,3.4,10.0,3.6,2656,Everton,1,1.129330,13,6,11,9,1.032325,1.409574,11.856460,4.111920,10.679454,4.446944,1.8,2.132290,14.4,5.0,11.0,5.6
2656,2656,2657,Tottenham,4,2.033150,11,7,5,3,1.742311,1.526763,13.475907,5.056770,11.409294,5.118106,1.6,1.556006,12.6,5.4,10.8,5.4,2657,Leeds,1,1.191510,19,2,7,12,1.233565,1.233433,11.759241,3.837085,12.175745,4.792627,1.2,1.247156,10.6,3.6,14.2,3.4
2657,2657,2658,West Ham,1,0.970352,16,3,10,5,1.308893,1.395155,11.728200,3.997522,9.044661,5.467341,1.4,1.384839,10.4,4.6,8.0,5.4,2658,Leicester,2,2.030130,13,4,8,3,1.288591,1.388841,10.911354,4.121708,10.978002,3.687683,1.2,1.789573,10.6,4.6,11.2,4.0
2658,2658,2659,Fulham,1,1.881670,10,3,10,4,1.354989,1.160589,11.088089,3.745470,11.385258,4.923457,2.0,1.230148,10.0,3.8,11.0,4.8,2659,Manchester United,2,2.747660,21,8,14,5,1.493654,1.960980,16.063134,5.735406,10.728826,5.575982,1.4,2.451314,20.0,6.4,10.2,6.4


In [ ]:
stat_df = stat_df.drop(['Unnamed: 0', 'GameId.1'], axis=1)

In [ ]:
stat_df.columns = ['GameId','HomeTeam','HomeGoals','HomexG','HomeShots','HomeShotsOT', 'HomeCorners', 'HomeFouls',
                   'HomeGoals_ema', 'HomexG_ema', 'HomeShots_ema', 'HomeShotsOT_ema', 'HomeCorners_ema', 'HomeFouls_ema',
                   'HomeGoals_5ma', 'HomexG_5ma', 'HomeShots_5ma', 'HomeShotsOT_5ma', 'HomeCorners_5ma', 'HomeFouls_5ma',
              'AwayTeam','AwayGoals','AwayxG','AwayShots', 'AwayShotsOT','AwayCorners', 'AwayFouls',
              'AwayGoals_ema', 'AwayxG_ema', 'AwayShots_ema', 'AwayShotsOT_ema', 'AwayCorners_ema', 'AwayFouls_ema',
              'AwayGoals_5ma', 'AwayxG_5ma', 'AwayShots_5ma', 'AwayShotsOT_5ma', 'AwayCorners_5ma', 'AwayFouls_5ma']

In [ ]:
cols = ['GameId','HomeTeam','HomeGoals','HomexG','HomeShots','HomeShotsOT', 'HomeCorners', 'HomeFouls',
                   'HomeGoals_ema', 'HomexG_ema', 'HomeShots_ema', 'HomeShotsOT_ema', 'HomeCorners_ema', 'HomeFouls_ema',
                   'HomeGoals_5ma', 'HomexG_5ma', 'HomeShots_5ma', 'HomeShotsOT_5ma', 'HomeCorners_5ma', 'HomeFouls_5ma',
              'AwayTeam','AwayGoals','AwayxG','AwayShots', 'AwayShotsOT','AwayCorners', 'AwayFouls',
              'AwayGoals_ema', 'AwayxG_ema', 'AwayShots_ema', 'AwayShotsOT_ema', 'AwayCorners_ema', 'AwayFouls_ema',
              'AwayGoals_5ma', 'AwayxG_5ma', 'AwayShots_5ma', 'AwayShotsOT_5ma', 'AwayCorners_5ma', 'AwayFouls_5ma']

In [ ]:
stat_df = stat_df[cols]
stat_df

,GameId,HomeTeam,HomeGoals,HomexG,HomeShots,HomeShotsOT,HomeCorners,HomeFouls,HomeGoals_ema,HomexG_ema,HomeShots_ema,HomeShotsOT_ema,HomeCorners_ema,HomeFouls_ema,HomeGoals_5ma,HomexG_5ma,HomeShots_5ma,HomeShotsOT_5ma,HomeCorners_5ma,HomeFouls_5ma,AwayTeam,AwayGoals,AwayxG,AwayShots,AwayShotsOT,AwayCorners,AwayFouls,AwayGoals_ema,AwayxG_ema,AwayShots_ema,AwayShotsOT_ema,AwayCorners_ema,AwayFouls_ema,AwayGoals_5ma,AwayxG_5ma,AwayShots_5ma,AwayShotsOT_5ma,AwayCorners_5ma,AwayFouls_5ma
0,1,Hull,2,0.740018,14,5,8,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Leicester,1,2.456310,18,5,17,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,West Bromwich Albion,1,0.684593,13,3,15,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crystal Palace,0,0.990062,14,4,12,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Everton,1,0.790063,12,6,10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tottenham,1,1.123480,13,4,14,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Burnley,0,1.049030,10,3,10,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Swansea,1,1.720890,17,9,14,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Middlesbrough,1,1.805230,12,2,18,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stoke,1,0.485626,12,1,14,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2656,Bournemouth,0,0.432084,7,2,12,3,1.044739,1.187594,10.191589,3.685048,10.069962,4.063956,1.2,1.160216,10.6,3.4,10.0,3.6,Everton,1,1.129330,13,6,11,9,1.032325,1.409574,11.856460,4.111920,10.679454,4.446944,1.8,2.132290,14.4,5.0,11.0,5.6
2656,2657,Tottenham,4,2.033150,11,7,5,3,1.742311,1.526763,13.475907,5.056770,11.409294,5.118106,1.6,1.556006,12.6,5.4,10.8,5.4,Leeds,1,1.191510,19,2,7,12,1.233565,1.233433,11.759241,3.837085,12.175745,4.792627,1.2,1.247156,10.6,3.6,14.2,3.4
2657,2658,West Ham,1,0.970352,16,3,10,5,1.308893,1.395155,11.728200,3.997522,9.044661,5.467341,1.4,1.384839,10.4,4.6,8.0,5.4,Leicester,2,2.030130,13,4,8,3,1.288591,1.388841,10.911354,4.121708,10.978002,3.687683,1.2,1.789573,10.6,4.6,11.2,4.0
2658,2659,Fulham,1,1.881670,10,3,10,4,1.354989,1.160589,11.088089,3.745470,11.385258,4.923457,2.0,1.230148,10.0,3.8,11.0,4.8,Manchester United,2,2.747660,21,8,14,5,1.493654,1.960980,16.063134,5.735406,10.728826,5.575982,1.4,2.451314,20.0,6.4,10.2,6.4


In [ ]:
stat_df = stat_df.filter(['GameId','HomeGoals_ema', 'AwayGoals_ema',
        'HomexG_ema', 'AwayxG_ema',
        'HomeShots_ema', 'AwayShots_ema', 'HomeShotsOT_ema',
        'AwayShotsOT_ema', 'HomeCorners_ema', 'AwayCorners_ema', 'HomeFouls_ema', 'AwayFouls_ema',
        'HomeGoals_5ma', 'AwayGoals_5ma', 'HomexG_5ma', 'AwayxG_5ma', 'HomeShots_5ma',
        'AwayShots_5ma', 'HomeShotsOT_5ma', 'AwayShotsOT_5ma', 'HomeCorners_5ma', 'AwayCorners_5ma', 'HomeFouls_5ma', 'AwayFouls_5ma'], axis=1)

In [ ]:
stat_df

,GameId,HomeGoals_ema,AwayGoals_ema,HomexG_ema,AwayxG_ema,HomeShots_ema,AwayShots_ema,HomeShotsOT_ema,AwayShotsOT_ema,HomeCorners_ema,AwayCorners_ema,HomeFouls_ema,AwayFouls_ema,HomeGoals_5ma,AwayGoals_5ma,HomexG_5ma,AwayxG_5ma,HomeShots_5ma,AwayShots_5ma,HomeShotsOT_5ma,AwayShotsOT_5ma,HomeCorners_5ma,AwayCorners_5ma,HomeFouls_5ma,AwayFouls_5ma
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2656,1.044739,1.032325,1.187594,1.409574,10.191589,11.856460,3.685048,4.111920,10.069962,10.679454,4.063956,4.446944,1.2,1.8,1.160216,2.132290,10.6,14.4,3.4,5.0,10.0,11.0,3.6,5.6
2656,2657,1.742311,1.233565,1.526763,1.233433,13.475907,11.759241,5.056770,3.837085,11.409294,12.175745,5.118106,4.792627,1.6,1.2,1.556006,1.247156,12.6,10.6,5.4,3.6,10.8,14.2,5.4,3.4
2657,2658,1.308893,1.288591,1.395155,1.388841,11.728200,10.911354,3.997522,4.121708,9.044661,10.978002,5.467341,3.687683,1.4,1.2,1.384839,1.789573,10.4,10.6,4.6,4.6,8.0,11.2,5.4,4.0
2658,2659,1.354989,1.493654,1.160589,1.960980,11.088089,16.063134,3.745470,5.735406,11.385258,10.728826,4.923457,5.575982,2.0,1.4,1.230148,2.451314,10.0,20.0,3.8,6.4,11.0,10.2,4.8,6.4


In [ ]:
#prófa crude strength á mörkum skoruðum og conceded. einhver leið að gera þetta að.
#líklega samt enn betra að gera þetta fyrir hvert ár fyrir sig, öruggara. færa þá ofar
df_final["HomeStrength"] = df_final["HTGS"] / df_final["HTGC"]
df_final["AwayStrength"] = df_final["ATGS"] / df_final["ATGC"]
df_final["HomexGStrength"] = df_final["HTxGS"] / df_final["HTxGC"]
df_final["AwayxGStrength"] = df_final["ATxGS"] / df_final["ATxGC"]

In [ ]:
df_final

,GameId,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,ATGS,HTGC,ATGC,HTxGS,ATxGS,HTxGC,ATxGC,winner,glicko_diff,target,HTP,ATP,Date,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HomeStrength,AwayStrength,HomexGStrength,AwayxGStrength
0,1,Hull,Leicester,2,1,0.740018,2.456310,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,W,0,0,13/08/16,0,14,18,5,5,5,3,8,17,0.222222,0.277778,0.523560,-----,-----,0,0,NaN,NaN,NaN,NaN
1,2,Crystal Palace,West Bromwich Albion,0,1,0.990062,0.684593,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,13/08/16,1,14,13,4,3,3,6,12,15,0.500000,0.303030,0.222222,-----,-----,0,0,NaN,NaN,NaN,NaN
2,3,Everton,Tottenham,1,1,0.790063,1.123480,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,13/08/16,0,12,13,6,4,5,6,10,14,0.312500,0.294118,0.416667,-----,-----,0,0,NaN,NaN,NaN,NaN
3,4,Burnley,Swansea,0,1,1.049030,1.720890,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,13/08/16,1,10,17,3,9,7,4,10,14,0.416667,0.303030,0.307692,-----,-----,0,0,NaN,NaN,NaN,NaN
4,5,Middlesbrough,Stoke,1,1,1.805230,0.485626,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,13/08/16,0,12,12,2,1,9,6,18,14,0.420168,0.312500,0.294118,-----,-----,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2656,Everton,Bournemouth,1,0,1.129330,0.432084,38,33,37,57,70,47.520506,39.651881,68.150778,66.472764,0,-324.743046,W,33,39,28/05/2023,1,13,7,6,2,9,3,11,12,0.666667,0.222222,0.153846,DLWDL,LLLWW,5,6,0.578947,0.528571,0.697285,0.596513
376,2657,Leeds,Tottenham,1,4,1.191510,2.033150,38,47,66,74,62,46.604455,55.800640,67.547883,50.469218,2,-585.675144,L,31,57,28/05/2023,2,19,11,2,7,12,3,7,5,0.363636,0.263158,0.434783,LDLLD,LLWLD,2,4,0.635135,1.064516,0.689947,1.105637
377,2658,Leicester,West Ham,2,1,2.030130,0.970352,38,49,41,67,53,46.956869,50.240380,58.818999,53.282043,0,-33.584483,W,31,40,28/05/2023,0,13,16,4,3,3,5,8,10,0.523560,0.263158,0.277778,DLLDD,WLWLL,3,6,0.731343,0.773585,0.798328,0.942914
378,2659,Manchester United,Fulham,2,1,2.747660,1.881670,38,56,54,42,51,69.153518,46.103307,48.405969,68.743151,0,73.719011,W,72,52,28/05/2023,1,21,10,8,3,5,4,14,10,0.666667,0.230947,0.166667,WWWLL,DWWLL,9,7,1.333333,1.058824,1.428616,0.670660


In [ ]:
#upp koma nokkur inf út af nýliðum, gefa þeim gildið 0.5, semsagt verra lið til að byrja með.
import numpy as np

df_final['HomeStrength'].replace(np.inf, 0.5, inplace=True)
df_final['AwayStrength'].replace(np.inf, 0.5, inplace=True)

In [ ]:
df_final

,GameId,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,ATGS,HTGC,ATGC,HTxGS,ATxGS,HTxGC,ATxGC,winner,glicko_diff,target,HTP,ATP,Date,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HomeStrength,AwayStrength,HomexGStrength,AwayxGStrength
0,1,Hull,Leicester,2,1,0.740018,2.456310,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,W,0,0,13/08/16,0,14,18,5,5,5,3,8,17,0.222222,0.277778,0.523560,-----,-----,0,0,NaN,NaN,NaN,NaN
1,2,Crystal Palace,West Bromwich Albion,0,1,0.990062,0.684593,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,13/08/16,1,14,13,4,3,3,6,12,15,0.500000,0.303030,0.222222,-----,-----,0,0,NaN,NaN,NaN,NaN
2,3,Everton,Tottenham,1,1,0.790063,1.123480,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,13/08/16,0,12,13,6,4,5,6,10,14,0.312500,0.294118,0.416667,-----,-----,0,0,NaN,NaN,NaN,NaN
3,4,Burnley,Swansea,0,1,1.049030,1.720890,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,13/08/16,1,10,17,3,9,7,4,10,14,0.416667,0.303030,0.307692,-----,-----,0,0,NaN,NaN,NaN,NaN
4,5,Middlesbrough,Stoke,1,1,1.805230,0.485626,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,13/08/16,0,12,12,2,1,9,6,18,14,0.420168,0.312500,0.294118,-----,-----,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2656,Everton,Bournemouth,1,0,1.129330,0.432084,38,33,37,57,70,47.520506,39.651881,68.150778,66.472764,0,-324.743046,W,33,39,28/05/2023,1,13,7,6,2,9,3,11,12,0.666667,0.222222,0.153846,DLWDL,LLLWW,5,6,0.578947,0.528571,0.697285,0.596513
376,2657,Leeds,Tottenham,1,4,1.191510,2.033150,38,47,66,74,62,46.604455,55.800640,67.547883,50.469218,2,-585.675144,L,31,57,28/05/2023,2,19,11,2,7,12,3,7,5,0.363636,0.263158,0.434783,LDLLD,LLWLD,2,4,0.635135,1.064516,0.689947,1.105637
377,2658,Leicester,West Ham,2,1,2.030130,0.970352,38,49,41,67,53,46.956869,50.240380,58.818999,53.282043,0,-33.584483,W,31,40,28/05/2023,0,13,16,4,3,3,5,8,10,0.523560,0.263158,0.277778,DLLDD,WLWLL,3,6,0.731343,0.773585,0.798328,0.942914
378,2659,Manchester United,Fulham,2,1,2.747660,1.881670,38,56,54,42,51,69.153518,46.103307,48.405969,68.743151,0,73.719011,W,72,52,28/05/2023,1,21,10,8,3,5,4,14,10,0.666667,0.230947,0.166667,WWWLL,DWWLL,9,7,1.333333,1.058824,1.428616,0.670660


In [ ]:
#sameinum þennan df við df_final
df_ready = pd.merge(left=df_final, right=stat_df,
                  left_on= ['GameId'], right_on=['GameId'],
                  how='left')

In [ ]:
df_ready

,GameId,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,ATGS,HTGC,ATGC,HTxGS,ATxGS,HTxGC,ATxGC,winner,glicko_diff,target,HTP,ATP,Date,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HomeStrength,AwayStrength,HomexGStrength,AwayxGStrength,HomeGoals_ema,AwayGoals_ema,HomexG_ema,AwayxG_ema,HomeShots_ema,AwayShots_ema,HomeShotsOT_ema,AwayShotsOT_ema,HomeCorners_ema,AwayCorners_ema,HomeFouls_ema,AwayFouls_ema,HomeGoals_5ma,AwayGoals_5ma,HomexG_5ma,AwayxG_5ma,HomeShots_5ma,AwayShots_5ma,HomeShotsOT_5ma,AwayShotsOT_5ma,HomeCorners_5ma,AwayCorners_5ma,HomeFouls_5ma,AwayFouls_5ma
0,1,Hull,Leicester,2,1,0.740018,2.456310,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,W,0,0,13/08/16,0,14,18,5,5,5,3,8,17,0.222222,0.277778,0.523560,-----,-----,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Crystal Palace,West Bromwich Albion,0,1,0.990062,0.684593,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,13/08/16,1,14,13,4,3,3,6,12,15,0.500000,0.303030,0.222222,-----,-----,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Everton,Tottenham,1,1,0.790063,1.123480,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,13/08/16,0,12,13,6,4,5,6,10,14,0.312500,0.294118,0.416667,-----,-----,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Burnley,Swansea,0,1,1.049030,1.720890,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2,0.000000,L,0,0,13/08/16,1,10,17,3,9,7,4,10,14,0.416667,0.303030,0.307692,-----,-----,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Middlesbrough,Stoke,1,1,1.805230,0.485626,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1,0.000000,D,0,0,13/08/16,0,12,12,2,1,9,6,18,14,0.420168,0.312500,0.294118,-----,-----,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2656,Everton,Bournemouth,1,0,1.129330,0.432084,38,33,37,57,70,47.520506,39.651881,68.150778,66.472764,0,-324.743046,W,33,39,28/05/2023,1,13,7,6,2,9,3,11,12,0.666667,0.222222,0.153846,DLWDL,LLLWW,5,6,0.578947,0.528571,0.697285,0.596513,1.044739,1.032325,1.187594,1.409574,10.191589,11.856460,3.685048,4.111920,10.069962,10.679454,4.063956,4.446944,1.2,1.8,1.160216,2.132290,10.6,14.4,3.4,5.0,10.0,11.0,3.6,5.6
2656,2657,Leeds,Tottenham,1,4,1.191510,2.033150,38,47,66,74,62,46.604455,55.800640,67.547883,50.469218,2,-585.675144,L,31,57,28/05/2023,2,19,11,2,7,12,3,7,5,0.363636,0.263158,0.434783,LDLLD,LLWLD,2,4,0.635135,1.064516,0.689947,1.105637,1.742311,1.233565,1.526763,1.233433,13.475907,11.759241,5.056770,3.837085,11.409294,12.175745,5.118106,4.792627,1.6,1.2,1.556006,1.247156,12.6,10.6,5.4,3.6,10.8,14.2,5.4,3.4
2657,2658,Leicester,West Ham,2,1,2.030130,0.970352,38,49,41,67,53,46.956869,50.240380,58.818999,53.282043,0,-33.584483,W,31,40,28/05/2023,0,13,16,4,3,3,5,8,10,0.523560,0.263158,0.277778,DLLDD,WLWLL,3,6,0.731343,0.773585,0.798328,0.942914,1.308893,1.288591,1.395155,1.388841,11.728200,10.911354,3.997522,4.121708,9.044661,10.978002,5.467341,3.687683,1.4,1.2,1.384839,1.789573,10.4,10.6,4.6,4.6,8.0,11.2,5.4,4.0
2658,2659,Manchester United,Fulham,2,1,2.747660,1.881670,38,56,54,42,51,69.153518,46.103307,48.405969,68.743151,0,73.719011,W,72,52,28/05/2023,1,21,10,8,3,5,4,14,10,0.666667,0.230947,0.166667,WWWLL,DWWLL,9,7,1.333333,1.058824,1.428616,0.670660,1.354989,1.493654,1.160589,1.960980,11.088089,16.063134,3.745470,

In [ ]:
#nokkrir hlutir sem þarf að skoða, nan í fyrstu leikjum nýliða en samt á vitlausum stað?, skoða is nan fjöldinn
#glicko rating er 0 í fyrstu umferð, mögulega í lagi en kannski betra að droppa
df_ready = df_ready.dropna(inplace=False)

In [ ]:
#Þetta er þá tilbúið til að halda áfram í eda og prófanir, loksins.
df_ready.to_csv('drive/My Drive/df_ready.csv')
df_ready = pd.read_csv('drive/My Drive/df_ready.csv')
df_ready

,Unnamed: 0,GameId,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HomexG,AwayxG,GW,HTGS,ATGS,HTGC,ATGC,HTxGS,ATxGS,HTxGC,ATxGC,winner,glicko_diff,target,HTP,ATP,Date,Half Time Result,HomeShots,AwayShots,HomeShotOT,AwayShotsOT,HomeFouls,AwayFouls,HomeCorners,AwayCorners,OddsH,OddsD,OddsA,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HomeStrength,AwayStrength,HomexGStrength,AwayxGStrength,HomeGoals_ema,AwayGoals_ema,HomexG_ema,AwayxG_ema,HomeShots_ema,AwayShots_ema,HomeShotsOT_ema,AwayShotsOT_ema,HomeCorners_ema,AwayCorners_ema,HomeFouls_ema,AwayFouls_ema,HomeGoals_5ma,AwayGoals_5ma,HomexG_5ma,AwayxG_5ma,HomeShots_5ma,AwayShots_5ma,HomeShotsOT_5ma,AwayShotsOT_5ma,HomeCorners_5ma,AwayCorners_5ma,HomeFouls_5ma,AwayFouls_5ma
0,10,11,Manchester United,Southampton,2,0,1.365530,0.613233,2,3,1,1,1,1.911050,0.937328,0.522847,0.180046,0,291.213945,W,3,1,19/08/16,0,12,13,5,1,4,0,11,8,0.653595,0.238095,0.133333,W----,D----,3,1,3.000000,1.000000,3.655085,5.206047,3.000000,1.000000,1.911050,0.937328,11.000000,24.000000,7.000000,6.000000,10.000000,8.000000,2.000000,6.000000,3.0,1.0,1.911050,0.937328,11.0,24.0,7.0,6.0,10.0,8.0,2.0,6.0
1,11,12,Stoke,Manchester City,1,4,1.310660,2.171980,2,1,2,1,1,0.485626,1.954790,1.805230,1.098770,2,-291.213945,L,1,3,20/08/16,2,8,12,3,6,7,7,12,13,0.181818,0.263158,0.578035,D----,W----,1,3,1.000000,2.000000,0.269011,1.779071,2.000000,1.000000,1.954790,0.485626,16.000000,12.000000,4.000000,1.000000,11.000000,14.000000,9.000000,6.000000,2.0,1.0,1.954790,0.485626,16.0,12.0,4.0,1.0,11.0,14.0,9.0,6.0
2,12,13,Burnley,Liverpool,2,0,0.197536,0.939025,2,0,4,1,3,1.049030,1.659100,1.720890,1.181970,0,-582.427889,W,0,3,20/08/16,0,3,26,2,5,1,12,14,5,0.153846,0.238095,0.636943,L----,W----,0,3,0.000000,1.333333,0.609586,1.403674,0.000000,4.000000,1.049030,1.659100,10.000000,16.000000,3.000000,7.000000,10.000000,17.000000,7.000000,4.000000,0.0,4.0,1.049030,1.659100,10.0,16.0,3.0,7.0,10.0,17.0,7.0,4.0
3,13,14,Swansea,Hull,0,2,1.141340,2.186550,2,1,2,0,1,1.720890,0.740018,1.049030,2.456310,2,0.000000,L,3,3,20/08/16,1,13,12,3,4,8,3,7,10,0.555556,0.277778,0.190476,W----,W----,3,3,0.500000,2.000000,1.640458,0.301272,1.000000,2.000000,1.720890,0.740018,17.000000,14.000000,9.000000,5.000000,14.000000,8.000000,4.000000,5.000000,1.0,2.0,1.720890,0.740018,17.0,14.0,9.0,5.0,14.0,8.0,4.0,5.0
4,14,15,Tottenham,Crystal Palace,1,0,2.256230,0.357943,2,1,0,1,1,1.123480,0.990062,0.790063,0.684593,0,291.213945,W,1,0,20/08/16,1,20,10,5,2,10,4,19,9,0.689655,0.210526,0.125000,D----,L----,1,0,1.000000,0.000000,1.422013,1.446205,1.000000,0.000000,1.123480,0.990062,13.000000,14.000000,4.000000,4.000000,14.000000,12.000000,6.000000,3.000000,1.0,0.0,1.123480,0.990062,13.0,14.0,4.0,4.0,14.0,12.0,6.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2577,2655,2656,Everton,Bournemouth,1,0,1.129330,0.432084,38,33,37,57,70,47.520506,39.651881,68.150778,66.472764,0,-324.743046,W,33,39,28/05/2023,1,13,7,6,2,9,3,11,12,0.666667,0.222222,0.153846,DLWDL,LLLWW,5,6,0.578947,0.528571,0.697285,0.596513,1.044739,1.032325,1.187594,1.409574,10.191589,11.856460,3.685048,4.111920,10.069962,10.679454,4.063956,4.446944,1.2,1.8,1.160216,2.132290,10.6,14.4,3.4,5.0,10.0,11.0,3.6,5.6
2578,2656,2657,Leeds,Tottenham,1,4,1.191510,2.033150,38,47,66,74,62,46.604455,55.800640,67.547883,50.469218,2,-585.675144,L,31,57,28/05/2023,2,19,11,2,7,12,3,7,5,0.363636,0.263158,0.434783,LDLLD,LLWLD,2,4,0.635135,1.064516,0.689947,1.105637,1.742311,1.233565,1.526763,1.233433,13.475907,11.759241,5.056770,3.837085,11.409294,12.175745,5.118106,4.792627,1.6,1.2,1.556006,1.247156,12.6,10.6,5.4,3.6,10.8,14.2,5.4,3.4
2579,2657,2658,Leicester,West Ham,2,1,2.030130,0.970352,38,49,41,67,53,46.956869,50.240380,58.818999,53.282043,0,-33.584483,W,31,40,28/05/2023,0,13,16,4,3,3,5,8,10,0.523560,0.263